In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [2]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [3]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [4]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [7]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [5]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.28923667
====> Test set loss: 1.2808, 63.5%
====> Epoch: 150 Average loss: 1.23414843
====> Test set loss: 1.2731, 60.5%
====> Epoch: 225 Average loss: 1.22775954
====> Test set loss: 1.2520, 63.0%
====> Epoch: 300 Average loss: 1.22451996
====> Test set loss: 1.2490, 63.0%
====> Epoch: 375 Average loss: 1.21432436
====> Test set loss: 1.2540, 62.0%
====> Epoch: 450 Average loss: 1.26265797
====> Test set loss: 1.2523, 62.0%
====> Epoch: 525 Average loss: 1.22552528
====> Test set loss: 1.2508, 62.0%
====> Epoch: 600 Average loss: 1.27635715
====> Test set loss: 1.2503, 62.0%
====> Epoch: 675 Average loss: 1.22551006
====> Test set loss: 1.2490, 62.5%
====> Epoch: 750 Average loss: 1.21764433
====> Test set loss: 1.2480, 62.5%
Training state:  False
Elapsed:  28.841941833496094
Complete set accuracy: 64.3%


### Runner

In [11]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(325, 350):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 300
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.33460891
====> Test set loss: 1.2224, 74.5%
====> Epoch: 150 Average loss: 1.27133732
====> Test set loss: 1.1800, 71.5%
====> Epoch: 225 Average loss: 1.25420722
====> Test set loss: 1.1756, 72.0%
====> Epoch: 300 Average loss: 1.25390979
====> Test set loss: 1.1655, 73.5%
====> Epoch: 375 Average loss: 1.23735471
====> Test set loss: 1.1636, 73.5%
====> Epoch: 450 Average loss: 1.24810542
====> Test set loss: 1.1633, 73.5%
====> Epoch: 525 Average loss: 1.26795135
====> Test set loss: 1.1634, 73.5%
====> Epoch: 600 Average loss: 1.26661420
====> Test set loss: 1.1627, 73.5%
====> Epoch: 675 Average loss: 1.22817767
====> Test set loss: 1.1627, 73.5%
====> Epoch: 750 Average loss: 1.24520123
====> Test set loss: 1.1626, 73.5%
Training state:  False
Complete set accuracy: 72.39999999999999%
Log accuracy: 69.39999999999999%
---- Done in  36.37389302253723  seconds

---- Running for model

====> Epoch: 675 Average loss: 1.09961659
====> Test set loss: 1.0899, 71.5%
====> Epoch: 750 Average loss: 1.14399066
====> Test set loss: 1.0898, 71.5%
Training state:  False
Complete set accuracy: 75.0%
Log accuracy: 74.1%
---- Done in  57.02410697937012  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.31303314
====> Test set loss: 1.3325, 59.5%
====> Epoch: 150 Average loss: 1.22384418
====> Test set loss: 1.2342, 70.5%
====> Epoch: 225 Average loss: 1.22971347
====> Test set loss: 1.2380, 69.5%
====> Epoch: 300 Average loss: 1.16342805
====> Test set loss: 1.2347, 69.5%
====> Epoch: 375 Average loss: 1.24775011
====> Test set loss: 1.2287, 69.5%
====> Epoch: 450 Average loss: 1.21405632
====> Test set loss: 1.2281, 69.5%
====> Epoch: 525 Average loss: 1.19255585
====> Test set loss: 1.2290, 69.5%
====> Epoch: 600 Average loss: 1.20765123
====> Test set loss: 1.2284, 69.5%
====> Epoch: 675 Average loss: 1.16515206
====> Test set loss: 1.2280, 6

====> Epoch: 450 Average loss: 1.20367826
====> Test set loss: 1.0803, 76.0%
====> Epoch: 525 Average loss: 1.14173083
====> Test set loss: 1.0796, 76.0%
====> Epoch: 600 Average loss: 1.13566555
====> Test set loss: 1.0776, 76.0%
====> Epoch: 675 Average loss: 1.15268233
====> Test set loss: 1.0773, 76.0%
====> Epoch: 750 Average loss: 1.13881647
====> Test set loss: 1.0752, 76.0%
Training state:  False
Complete set accuracy: 77.0%
Log accuracy: 74.7%
---- Done in  56.449243783950806  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.24876652
====> Test set loss: 1.1552, 72.0%
====> Epoch: 150 Average loss: 1.20322392
====> Test set loss: 1.1160, 73.0%
====> Epoch: 225 Average loss: 1.20524515
====> Test set loss: 1.1134, 73.5%
====> Epoch: 300 Average loss: 1.14698252
====> Test set loss: 1.1106, 72.5%
====> Epoch: 375 Average loss: 1.19640746
====> Test set loss: 1.1034, 72.0%
====> Epoch: 450 Average loss: 1.19685847
====> Test set loss: 1

====> Epoch: 225 Average loss: 1.22777835
====> Test set loss: 1.2286, 62.5%
====> Epoch: 300 Average loss: 1.24096246
====> Test set loss: 1.2266, 63.0%
====> Epoch: 375 Average loss: 1.25576215
====> Test set loss: 1.2264, 62.5%
====> Epoch: 450 Average loss: 1.23356166
====> Test set loss: 1.2275, 62.5%
====> Epoch: 525 Average loss: 1.24950181
====> Test set loss: 1.2285, 62.5%
====> Epoch: 600 Average loss: 1.26337518
====> Test set loss: 1.2278, 62.5%
====> Epoch: 675 Average loss: 1.21066835
====> Test set loss: 1.2278, 62.5%
====> Epoch: 750 Average loss: 1.20546919
====> Test set loss: 1.2269, 62.5%
Training state:  False
Complete set accuracy: 70.1%
Log accuracy: 70.5%
---- Done in  57.54643201828003  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.23545908
====> Test set loss: 1.1777, 75.5%
====> Epoch: 150 Average loss: 1.22114753
====> Test set loss: 1.1126, 79.0%
====> Epoch: 225 Average loss: 1.21144165
====> Test set loss: 1.10

====> Epoch: 75 Average loss: 1.27355229
====> Test set loss: 1.2102, 74.0%
====> Epoch: 150 Average loss: 1.19620341
====> Test set loss: 1.1533, 74.0%
====> Epoch: 225 Average loss: 1.20847197
====> Test set loss: 1.1580, 75.0%
====> Epoch: 300 Average loss: 1.20136867
====> Test set loss: 1.1599, 74.5%
====> Epoch: 375 Average loss: 1.18048703
====> Test set loss: 1.1631, 74.0%
====> Epoch: 450 Average loss: 1.18603839
====> Test set loss: 1.1627, 74.0%
====> Epoch: 525 Average loss: 1.19577722
====> Test set loss: 1.1624, 74.0%
====> Epoch: 600 Average loss: 1.17439228
====> Test set loss: 1.1620, 74.0%
====> Epoch: 675 Average loss: 1.16408560
====> Test set loss: 1.1616, 74.0%
====> Epoch: 750 Average loss: 1.22389509
====> Test set loss: 1.1617, 74.0%
Training state:  False
Complete set accuracy: 74.0%
Log accuracy: 71.1%
---- Done in  56.75322699546814  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.27686825
====> Test set loss: 1.2412, 7

...

====> Epoch: 300 Average loss: 1.17880829
====> Test set loss: 1.2710, 64.5%
====> Epoch: 375 Average loss: 1.20644039
====> Test set loss: 1.2715, 64.5%
====> Epoch: 450 Average loss: 1.22673784
====> Test set loss: 1.2717, 65.0%
====> Epoch: 525 Average loss: 1.19194501
====> Test set loss: 1.2717, 65.0%
====> Epoch: 600 Average loss: 1.15289601
====> Test set loss: 1.2718, 65.0%
====> Epoch: 675 Average loss: 1.15828856
====> Test set loss: 1.2719, 65.0%
====> Epoch: 750 Average loss: 1.15267383
====> Test set loss: 1.2719, 65.0%
Training state:  False
Complete set accuracy: 72.0%
Log accuracy: 70.7%
---- Done in  56.24994087219238  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.22977405
====> Test set loss: 1.1541, 73.0%
====> Epoch: 150 Average loss: 1.13894264
====> Test set loss: 1.0683, 75.5%
====> Epoch: 225 Average loss: 1.16748809
====> Test set loss: 1.0672, 75.5%
====> Epoch: 300 Average loss: 1.16559355
====> Test set loss: 1.0627, 

====> Epoch: 75 Average loss: 1.26695765
====> Test set loss: 1.1921, 68.0%
====> Epoch: 150 Average loss: 1.17419536
====> Test set loss: 1.1300, 70.0%
====> Epoch: 225 Average loss: 1.17890018
====> Test set loss: 1.1228, 69.5%
====> Epoch: 300 Average loss: 1.15388194
====> Test set loss: 1.1252, 69.5%
====> Epoch: 375 Average loss: 1.19793150
====> Test set loss: 1.1182, 71.0%
====> Epoch: 450 Average loss: 1.14944444
====> Test set loss: 1.1191, 71.0%
====> Epoch: 525 Average loss: 1.14185113
====> Test set loss: 1.1201, 71.0%
====> Epoch: 600 Average loss: 1.17017701
====> Test set loss: 1.1214, 70.5%
====> Epoch: 675 Average loss: 1.14092472
====> Test set loss: 1.1216, 70.5%
====> Epoch: 750 Average loss: 1.21024150
====> Test set loss: 1.1215, 70.5%
Training state:  False
Complete set accuracy: 75.4%
Log accuracy: 69.39999999999999%
---- Done in  66.16309309005737  seconds

---- Running for model name:  D_mild_nadd_lin
====> Epoch: 75 Average loss: 1.27478902
====> Test set lo

====> Epoch: 750 Average loss: 1.08323099
====> Test set loss: 1.0819, 73.5%
Training state:  False
Complete set accuracy: 78.7%
Log accuracy: 74.3%
---- Done in  73.5171070098877  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.26477985
====> Test set loss: 1.2000, 75.0%
====> Epoch: 150 Average loss: 1.20046233
====> Test set loss: 1.1594, 69.5%
====> Epoch: 225 Average loss: 1.17406919
====> Test set loss: 1.1599, 70.0%
====> Epoch: 300 Average loss: 1.18283808
====> Test set loss: 1.1595, 69.5%
====> Epoch: 375 Average loss: 1.18154908
====> Test set loss: 1.1613, 70.0%
====> Epoch: 450 Average loss: 1.18550598
====> Test set loss: 1.1597, 70.0%
====> Epoch: 525 Average loss: 1.18436822
====> Test set loss: 1.1589, 70.0%
====> Epoch: 600 Average loss: 1.12298059
====> Test set loss: 1.1590, 70.0%
====> Epoch: 675 Average loss: 1.17041398
====> Test set loss: 1.1590, 70.0%
====> Epoch: 750 Average loss: 1.16920606
====> Test set loss: 1.1

====> Epoch: 525 Average loss: 1.19965444
====> Test set loss: 1.1362, 72.0%
====> Epoch: 600 Average loss: 1.21140948
====> Test set loss: 1.1357, 72.0%
====> Epoch: 675 Average loss: 1.17630477
====> Test set loss: 1.1359, 72.0%
====> Epoch: 750 Average loss: 1.14566873
====> Test set loss: 1.1357, 72.0%
Training state:  False
Complete set accuracy: 77.0%
Log accuracy: 73.0%
---- Done in  68.69234371185303  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.22668797
====> Test set loss: 1.1730, 73.5%
====> Epoch: 150 Average loss: 1.18958944
====> Test set loss: 1.1151, 72.5%
====> Epoch: 225 Average loss: 1.20842103
====> Test set loss: 1.1085, 73.0%
====> Epoch: 300 Average loss: 1.16550863
====> Test set loss: 1.1061, 73.0%
====> Epoch: 375 Average loss: 1.17262856
====> Test set loss: 1.1012, 73.5%
====> Epoch: 450 Average loss: 1.19269239
====> Test set loss: 1.1007, 73.5%
====> Epoch: 525 Average loss: 1.17373783
====> Test set loss: 1.09

KeyboardInterrupt: 